In [1]:
import random

In [2]:
class State():
    def __init__(self, name, value, reciprocal, src):
        self._name = name
        self._value = value
        self._src = src
        self._reciprocal = reciprocal

    def get_name(self):
        return self._name
        
    def get_value(self):
        return self._value
    
    def round_down(self, value=1):
        self._reciprocal -= value
        if self._reciprocal <= 0:
            return False
        return True
    
    def set_state(self, value, reciprocal, src):
        self._value = value
        self._src = src
        self._reciprocal = reciprocal

In [3]:
class Character():
    def __init__(self,
                 life,
                 attack,
                 defence,
                 speed):
        self._life = life
        self._attack = attack
        self._defence = defence
        self._speed = speed
        self._states = []
        self._action_count = 0
        
    def action(self, target):
        self._action_count += 1
        for state in self._states:
            if state.get_name() == 'paralysis':
                return False
        for state in self._states:
            if state.get_name() == 'dominate':
                self._normal_attack(self)
        return True
            
    
    def _normal_attack(self, target):
        attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        target.damaged(attack)
        
    def _active_skill(self, target):
        return
    
    def _passive_skill(self, target):
        return
    
    def damaged(self, value, true_damage = False):
        if true_damage:
            self._life -= value
        else:
            defence = self._defence
            for state in self._states:
                if state.get_name() == 'defence change':
                    defence += state.get_value()
            damage = value - defence
            if damage <0:
                damage = 0
            self._life -= damage
        
    def get_speed(self):
        speed = self._speed
        for state in self._states:
            if state.get_name() == 'speed change':
                speed += state.get_value()
        return speed
    
    def get_life(self):
        return self._life
    
    def set_state(self, name, value, reciprocal, src, independent=False):
        if independent:
            self._states.append(State(name, value, reciprocal, src))
        else:
            for state in self._states:
                if state.get_name() == name:
                    state.set_state(value, reciprocal, src)
                    return
            
    def round_end(self):
        new_state = []
        for state in self._states:
            if state.round_down():
                new_state.append(state)
        self._states = new_state

In [24]:
class Thelema(Character):
    def __init__(self,
                 life=100,
                 attack=19,
                 defence=9,
                 speed=23):
        super(Thelema, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 2 == 0:
            self._active_skill(target)
        self._normal_attack(target)
        self._passive_skill(target)
    
    def _active_skill(self, target):
        target.damaged(16)
        
    def _passive_skill(self, target):
        if target != self and random.random() < 0.2:
            target.set_state('dominate', None, 2, self)

In [5]:
class Songque(Character):
    def __init__(self,
                 life=100,
                 attack=19,
                 defence=9,
                 speed=20):
        super(Songque, self).__init__(life, attack, defence, speed)
        self._shadow = 0
        
    def action(self, target):
        if not super().action(target):
            return
        if self._action_count % 3 == 0:
            self._active_skill(target)
        self._normal_attack(target)
        self._passive_skill(target)
        
    def _normal_attack(self, target):
        attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        attack += 4*self._shadow
        target.damaged(attack)
        
    def _active_skill(self, target):
        target.damaged(15 + 4*self._shadow)
    
    def _passive_skill(self, target):
        if random.random() < 0.25 and self._shadow < 3:
            self._shadow += 1
            target.set_state('defence change', -4, 99, self, independent=True)
            target.set_state('speed change', -2, 99, self, independent=True)

In [21]:
class Dreamseeker(Character):
    def __init__(self,
                 life=100,
                 attack=16,
                 defence=8,
                 speed=21):
        super(Dreamseeker, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        self._passive_skill(target)
        if self._action_count % 3 ==0:
            self._active_skill(target)
        else:
            self._normal_attack(target)
        
        
    def _active_skill(self, target):
        attack = self._attack
        for state in self._states:
            if state.get_name() == 'attack change':
                attack += state.get_value()
        attack = 1.3*attack
        target.damaged(int(attack), true_damage=True)
    
    def _passive_skill(self, target):
        if random.random() < 0.7:
            if self._life < 50:
                self.set_state('attack change', 30-self._attack, 1, self)
            else:
                self.set_state('attack change', 24-self._attack, 1, self)

In [7]:
class Senadina(Character):
    def __init__(self,
                 life=100,
                 attack=21,
                 defence=6,
                 speed=23):
        super(Senadina, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        self._normal_attack(target)
        self._passive_skill(target)
        if self._action_count % 3 == 0:
            self._active_skill(target)
            self._passive_skill(target)
        
        
    def _active_skill(self, target):
        self.set_state("lightning", None, 2, self)
        target.damaged(15)
        target.set_state("defence change", -5, 2, self)
    
    def _passive_skill(self, target):
        for state in self._states:
            if state.get_name() == 'lightning':
                effect_count = random.randint(1, 3)
                target.damaged(effect_count*3, true_damage=True)
                if effect_count == 3:
                    target.set_state('paralysis', None, 2, self)

In [ ]:
class Seraphim(Character):
    def __init__(self,
                 life=100,
                 attack=20,
                 defence=8,
                 speed=15):
        super(Seraphim, self).__init__(life, attack, defence, speed)
        
    def action(self, target):
        if not super().action(target):
            return
        

In [15]:
def run(p1, p2):
    while p1.get_life() > 0 and p2.get_life() > 0:
        if p1.get_speed() > p2.get_speed():
            p1.action(p2)
            p1.round_end()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p2.action(p1)
            p2.round_end()
        else:
            p2.action(p1)
            p2.round_end()
            if p1.get_life() <= 0 or p2.get_life() <= 0:
                break
            p1.action(p2)
            p1.round_end()
    if p1.get_life() <= 0:
        return 0
    else:
        return 1

In [25]:
count = 0
for i in range(100000):
    count += run(Thelema(), Senadina())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*3.1)/100000,100000-count,(100000-count)*1.5/100000))

28610(0.8869) 71390(1.0709)


In [26]:
count = 0
for i in range(50000):
    count += run(Thelema(), Senadina())
for i in range(50000):
    count += 1 - run(Senadina(), Thelema())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*3.1)/100000,100000-count,(100000-count)*1.5/100000))

48800(1.5128) 51200(0.7680)


In [27]:
count = 0
for i in range(100000):
    count += 1 - run(Senadina(), Thelema())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*3.1)/100000,100000-count,(100000-count)*1.5/100000))

68968(2.1380) 31032(0.4655)


In [28]:
count = 0
for i in range(100000):
    count += run(Senadina(), Dreamseeker())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.4)/100000,100000-count,(100000-count)*3.7/100000))

72888(1.0204) 27112(1.0031)


In [29]:
count = 0
for i in range(100000):
    count += run(Thelema(), Dreamseeker())
print("{:d}({:.4f}) {:d}({:.4f})".format(count, (count*1.4)/100000,100000-count,(100000-count)*3.2/100000))

49031(0.6864) 50969(1.6310)
